# Konvertering fra korpus i pdf til et vasket korpus i ren tekst

## Etablering av et pdf-korpus

På [reiselivsforum.no](reiselivsforum.no) publiseres det avgjørelser fra Transportklagenemnda og Pakkereisenemnda.  Vi har lastet alle avgjørelsen i pdf-format. Nettsiden tilbyr ikke samlet nedlasting; det er nødvendig å lage et program som laster ned alle avgjørelser. Vi har utviklet et skript i Python modulen Scrapy som har lastet ned filer. Alle pdf-er er lastet ned til samme mappe. Pdf-ene er lagret med saksnummeret som filnavn. Søkemotoren på reiselivsforum.no oppgir totalt antall vedtak til 12853 pr. 15. oktober 2023. Skriptet lastet ned 12595 vedtak. Hva diskrepansen på 258 vedtak skyldes, vet vi ikke. Mange av de eldre vedtakene har et rettighetsforbehold fra Lovdata. Forbeholdet ser ut til å være en standardformulering og er den samme i alle vedtakene med forbehold:
> De publiserte nemndsavgjørelsene er bearbeidet av stiftelsen Lovdata, som innehar rettighetene til dokumentene. Utnyttelse av nemndsavgjørelsene til forlagsvirksomhet, distribusjon, drift av søkbare databaser eller opplæringsvirksomhet krever særskilt avtale med Lovdata. Lovdata avgjør i tvilstilfelle hva som faller inn under nevnte kategorier.

Her kan det være verdt å merke seg at forbeholdet står i vedtakene som er tilgjengelig for nedlasting nemndenes egen nettside. 

For å ikke komme i konflikt med rettighetsreservasjonen, fjerner vi de aktuelle vedtakene fra tekstkorpuset. Tilsammen fjerner vi da 4768 vedtak. De fordeler seg som følger.

| Nemnd                   | Vedtak med rettighetsforbehold |
|:------------------------| ------------------------------:|
|Pakkereiseklagenemnda    | 1734                           |
|Transportklagenemnda-Tog | 171                            |
|Transportklagenmnda-Fly  | 2863                           |
|**Sum:**                 |**4768**                        |

Vi står da igjen med 7827 vedtak fra 2017 til 2023. Søk i de gjenværende vedtakene bekrefter at de ikke har den samme rettighetsreservasjonen.

**Her må vi inn med en oversikt over antall vedtak pr. år. pr. nemnd. Kanskje det blir mulig å si noe om de 258 vi manglet også.**

## Konverting av pdf-er til ren tekst

For at avgjørelsene skal kunne brukes videre og behandles maskinelt, må pdf-ene konverteres til ren tekst. Nøyaktig konvertering til ren tekst er viktig for å kunne identifisere de ulike delene av vedtaket. Særlig kommer dette godt med når vi skal identifisere mindretallsvotum, og når vi skal eksperimentere med å bruke ulike deler av dokumentet som kontekst for språkmodellen (se under).

Vi prøver først å konvertere et vedtak i pdf til tekst med PyPDF2. 

In [14]:
import os
import PyPDF2

def pdf_to_text_pyPDF2(pdf_path):
    # Open the PDF file
    with open(pdf_path, 'rb') as file:
        # Create a PDF file reader
        pdf_reader = PyPDF2.PdfReader(file)
        
        # Loop through each page in the PDF file
        text = ""
        for page_num in range(len(pdf_reader.pages)):
            # Extract text from each page
            page = pdf_reader.pages[page_num]
            text += page.extract_text()
    
    return text

# Specify the path to your PDF file
path_to_pdf = "./pdfs/2016-00003.pdf"
text = pdf_to_text_pyPDF2(path_to_pdf)
print(text)
print(type(text))


Vedtak i Pakkereisenemnda
Sammendrag
Krav om dekning av merutgifter da togreisen "Norway in a nutshell" ble kansellert.
Dato
30.04.2017
Saksnummer
2016-00003
Tjenesteytere
Fjord Tours
Klager har i det vesentlige anført
Klager bestilte reisen "Norway in a nutshell" fra Oslo 15. - 16. oktober 2016 hos Fjord
Tours.
Først da klager ankom Myrdal, fant de ut at togreisen "Norway in a nutshell" var
kansellert. Klager ble ikke informert om dette.  
Hverken Fjord Tours eller NSB vil dekke klagers merutgifter til hotell, ﬂy, buss, taxi, tog og
tapt arbeidsfortjeneste.
Klager har fått refundert prisen for "Norway in a nutshell" med kr. 5.300, men krever å
få dekket de øvrige merutgiftene på kr. 6.270,57 fra Fjord Tours i og med at han bestilte
reisen hos Fjord Tours og ikke direkte hos NSB.
Det vises for øvrig til korrespondanse fra klager.
Tjenesteyterne har i det vesentlige anført
Fjord Tours har anført at saken er avsluttet fra Fjord Tours side da de i mail av 18.10.2016
informerte om at de vi

Som vi ser mangler det en del struktur. Det er vanskelig å identifisere overskrifter og avsnitt. Vi prøver derfor å konvertere pdf-ene til ren tekst med **tika**.

In [31]:
from tika import parser

parsed_pdf = parser.from_file("./pdfs/2016-00003.pdf")
vedtak_txt = parsed_pdf['content']
#print(repr(vedtak_txt))
print(vedtak_txt)





































Vedtak i Pakkereisenemnda

Sammendrag

Krav om dekning av merutgifter da togreisen "Norway in a nutshell" ble kansellert.

Dato
30.04.2017

Saksnummer
2016-00003

Tjenesteytere
Fjord Tours

Klager har i det vesentlige anført

Klager bestilte reisen "Norway in a nutshell" fra Oslo 15. - 16. oktober 2016 hos Fjord
Tours.

Først da klager ankom Myrdal, fant de ut at togreisen "Norway in a nutshell" var
kansellert. Klager ble ikke informert om dette.  

Hverken Fjord Tours eller NSB vil dekke klagers merutgifter til hotell, fly, buss, taxi, tog og
tapt arbeidsfortjeneste.

Klager har fått refundert prisen for "Norway in a nutshell" med kr. 5.300, men krever å
få dekket de øvrige merutgiftene på kr. 6.270,57 fra Fjord Tours i og med at han bestilte
reisen hos Fjord Tours og ikke direkte hos NSB.

Det vises for øvrig til korrespondanse fra klager.

Tjenesteyterne har i det vesentlige anført
Fjord Tours har anført at saken er avsluttet fra Fjord Tours side d

In [ ]:
Resultatet er bedre. Nå er mye struktur i bevart. 

In [ ]:
vedtak = vedtak_txt.replace('\n\n\n\n', '\n')

In [3]:
from tika import parser
import os
vedtak_alle = []
pdf_directory = './pdfs_small/'

for filename in os.listdir(pdf_directory):
    if filename.endswith('.pdf'):
        pdf_path = os.path.join(pdf_directory, filename)
        parsed_pdf = parser.from_file(pdf_path)
        vedtak_txt = parsed_pdf['content']
        vedtak_alle.append(vedtak_txt)     

In [10]:
print(vedtak_alle[1])




































Vedtak i Transportklagenemnda - Fly

Sammendrag

Krav om refusjon av enten opprinnelige eller
nyinnkjøpte retur-billetter, samt omkostninger.

Dato
06.04.2022

Saksnummer
2021-01887

Tjenesteytere
Turkish Airlines

Klager har i det vesentlige anført

Klager og hans medreisende hadde bestilt en tur-retur reise med Turkish Airlines fra Oslo
til Istanbul, med avreise den 26. oktober og retur den 30. oktober 2021. Klager rakk ikke
fly fra Oslo grunnet en trafikkulykke. Klager kontaktet Turskish Airlines kundeservice som
kunne tilby nye billetter til ca.13000 kroner per person. De informerte også om at klager
hadde en gyldig returbillett. Klager valgte da selv å bestille ny separat billett for utreise
den 27. oktober, og la returen den 30. oktober stå som opprinnelig bestilt.

Klager mottok e-post fra selskapet den 29. oktober, som viste at de var sjekket inn på
reisen fra Istanbul til Oslo den 30. oktober 2021. 
Da de skulle sjekke inn bagasjen i Istanbul

In [ ]:
for vedtak in vedtak_alle:
    print(vedtak)

In [ ]:
from IPython.display import display, Markdown
import json
import openai
import os
openai_api_key = os.getenv('OPENAI_API_KEY')

In [ ]:
# Define the string to search for
search_string = "Nemndas representanter"

# Split the text into lines
lines = text.split('\n')

# Initialize a flag to track if we should skip lines
skip_lines = False

# Initialize an empty list to store the filtered lines
filtered_lines = []

# Loop through each line in the text
for line in lines:
    # If we should skip lines, continue to the next line
    if skip_lines:
        continue

    # Check if the line starts with the search string
    if line.startswith(search_string):
        # If it does, set the skip_lines flag to True
        skip_lines = True
    else:
        # If it doesn't, add the line to the filtered_lines list
        filtered_lines.append(line)

# Join the filtered lines back together into a single text
data = '\n'.join(filtered_lines)

# Now, the filtered_text variable contains the text with lines starting with "Nemndas representanter" and the subsequent lines removed.


In [ ]:
# Printing of content  
print(data) 

In [ ]:
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer, LTChar

def extract_text_with_font(pdf_path):
    text_info = []

    for page_layout in extract_pages(pdf_path):
        for element in page_layout:
            if isinstance(element, LTTextContainer):
                for text_line in element:
                    for char in text_line:
                        if isinstance(char, LTChar):
                            text_info.append({
                                "text": char.get_text(),
                                "font_name": char.fontname,
                                "size": char.size,
                            })

    return text_info

def identify_bold_text(text_info):
    bold_texts = []
    current_bold_text = ""
    is_previous_char_bold = False

    for char_info in text_info:
        is_current_char_bold = "Bold" in char_info["font_name"]

        # If the current character is bold, we add it to the current bold text string
        if is_current_char_bold:
            current_bold_text += char_info["text"]
        # If the current character is not bold but the previous one was,
        # we add the current bold text to the list and reset it
        elif not is_current_char_bold and is_previous_char_bold and current_bold_text:
            bold_texts.append(current_bold_text)
            current_bold_text = ""

        is_previous_char_bold = is_current_char_bold

    # If the last character was bold, we add the remaining bold text to the list
    if current_bold_text:
        bold_texts.append(current_bold_text)

    return bold_texts


def main(pdf_path):
    text_info = extract_text_with_font(pdf_path)
    bold_texts = identify_bold_text(text_info)

    print("Identified bold texts:")
    for text in bold_texts:
        print(text)  # This will print each separate bold text as a string in the list


if __name__ == "__main__":
    pdf_path = "./pdfs_small/2021-01885.pdf"  # replace with your PDF file path
    main(pdf_path)


In [ ]:
import os
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer, LTChar, LTTextLine

def extract_text_with_font(pdf_path):
    text_info = []

    for page_layout in extract_pages(pdf_path):
        for element in page_layout:
            if isinstance(element, LTTextContainer):
                for text_line in element:
                    if isinstance(text_line, LTTextLine):
                        line_y = text_line.bbox[3]
                        for char in text_line:
                            if isinstance(char, LTChar):
                                text_info.append({
                                    "text": char.get_text(),
                                    "font_name": char.fontname,
                                    "size": char.size,
                                    "line_y": line_y,
                                })
    return text_info

def should_include(text):
    """
    Define your checks in this function. 
    Return True if the text should be included, False otherwise.
    """
    # Strings not starting with a capital letter should not be included
#    if not text[0].isupper():
#        return False
#
#    # Strings with 7 or more words should not be included
#    if len(text.split()) >= 7:
#        return False

    return True


def clean_text(text):
    """
    Clean the text based on the specified requirements.
    """
    # Strip ":" and any following characters
#    if ":" in text:
#        text = text.split(":")[0]

    return text.strip()  # Also strip whitespace from the beginning and end

def identify_bold_text(text_info):
    bold_texts = set()
    current_bold_text = ""
    previous_y = None

    for char_info in text_info:
        current_y = char_info["line_y"]

        if previous_y is not None and abs(current_y - previous_y) > 1:
            if current_bold_text:
                current_bold_text = clean_text(current_bold_text)
                if should_include(current_bold_text):
                    bold_texts.add(current_bold_text)
                current_bold_text = ""

        is_bold = "Bold" in char_info["font_name"]

        if is_bold:
            current_bold_text += char_info["text"]
        elif not is_bold and current_bold_text:
            current_bold_text = clean_text(current_bold_text)
            if should_include(current_bold_text):
                bold_texts.add(current_bold_text)
            current_bold_text = ""

        previous_y = current_y

    if current_bold_text:
        current_bold_text = clean_text(current_bold_text)
        if should_include(current_bold_text):
            bold_texts.add(current_bold_text)

    return bold_texts

def main(folder_path):
    overskrifter = set()
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".pdf"):
            pdf_path = os.path.join(folder_path, file_name)
            text_info = extract_text_with_font(pdf_path)
            bold_texts = identify_bold_text(text_info)

            overskrifter.update(bold_texts)

    for overskrift in sorted(overskrifter):
        print(overskrift)

if __name__ == "__main__":
    folder_path = "pdfs_medium/"
    main(folder_path)
